In [54]:
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 加载数据

In [55]:
# step1 加载文件
with open('../../data/切词.txt', 'r', encoding='utf-8') as file:
  docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])

vectorizer_model = None

条数:  1000
预览第一条:  文旅文 创看 洛阳 河南省 文旅文创 发展 大会 本次 大会 安排 项目 签约 主要 方面 内容 一是 文旅 产业 项目 签约 截至 目前 梳理 重点 文旅 项目 投资总额 525.6 亿元 遴选 重大项目 进行 现场 签约 投资总额 365.8 亿元 项目 包括 文物 数字化 开发 文化 创意 园区 建设 文化 项目 涵盖 旅游 度假区 建设 旅游 酒店 民宿 打造 旅游 项目 既有 旅游 景区 开发 商旅 综合体 建设 传统 业态 项目 宇宙 基地 沉浸 演艺 业态 项目 充分体现 我省 文化 旅游 发展 特点 趋势 二是 引客 入豫 项目 签约 主要 我省 文旅 部门 文旅 企业 头部 旅行 知名 OTA 平台 重点 客源地 文旅 部门 签订 引客 入豫 协议 持续 拓展 省外 客源 市场



# 创建

In [56]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="bert-base-chinese") # 使用bert-base-chinese
# embedding_model = pipeline("feature-extraction", model="hfl/chinese-bert-wwm") # 使用bert-base-chinese
embeddings = np.load('../../data/embedding_bbc.npy') # 使用bert-base-chinese向量
print(embeddings.shape)

# 2. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

# 3. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 4. 创建HDBSCAN聚类模型
# 如果要建设离群值，可以减小下面两个参数
# https://hdbscan.readthedocs.io/en/latest/faq.html
hdbscan_model = HDBSCAN(
  min_cluster_size=20,
  min_samples=5,
)

# 5. 创建CountVectorizer模型
vectorizer_model = CountVectorizer(stop_words=['洛阳', '旅游', '文化'])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1000, 768)


In [57]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,247,-1_景区_城市_河南_中国,"[景区, 城市, 河南, 中国, 游客, 历史, 洛阳市, 遗址, 博物馆, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,162,0_景区_发展_项目_建设,"[景区, 发展, 项目, 建设, 国家, 河南省, 活动, 河南, 工作, 洛阳市]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
2,1,98,1_活动_景区_免费_高速,"[活动, 景区, 免费, 高速, 门票, 时间, 白云山, 地点, 栾川, 安全可靠]",[洛阳 身边 自驾游 栾川 高速 免费 答疑 自驾游 栾川 高速 费全免 问题 需要 了解 ...
3,2,81,2_石窟_中国_龙门石窟_艺术,"[石窟, 中国, 龙门石窟, 艺术, 世界, 莫高窟, 文化遗产, 朝代, 造像, 少林寺]",[旅行 洛阳 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 ...
4,3,66,3_王府_竹海_河南_旅行,"[王府, 竹海, 河南, 旅行, 景区, 安阳, 瀑布, 旅游区, 栾川, 度假]",[每年 中国 旅游 一天 选择 出游 可能 今年 旅行 计划 实现 一定 提前 规划 今天 ...
5,4,52,4_晚会_广场_活动_文旅,"[晚会, 广场, 活动, 文旅, 体验, 话题, 河南, 历史, 沉浸, 隋唐洛阳城]",[中国 旅游 泉州 举行 多项 文旅 活动 鲤城区 城南 庙会 再次 开启 传统工艺 传统 ...
6,5,44,5_建设_机场_项目_规划,"[建设, 机场, 项目, 规划, 铁路, 自驾车, 黄河, 发展, 国家, 航线]",[国家 发展 改革 印发 汉江 生态 经济带 发展 规划 通知 汉江 生态 经济带 规划 范...
7,6,42,6_地方_很多_一天_一点,"[地方, 很多, 一天, 一点, 已经, 时间, 古城, 酒店, 西安, 感受]",[洛阳 值得 一去 地方 简单 下来 洛阳 旅游 攻略 建议 夏天 洛阳 洛阳 市区 主要 ...
8,7,42,7_泉州_旅行_洛阳桥_历史,"[泉州, 旅行, 洛阳桥, 历史, 济南, 中国, 古城, 沧州, 黄陂, 洛邑]",[海丝 泉州 夏天 清晨 洛阳桥 洛阳 日出 夏天 清晨 来到 泉州 洛阳桥 沐浴 晨光 长...
9,8,41,8_旅客_年票_郑州_客流,"[旅客, 年票, 郑州, 客流, 列车, 出行, 客运, 高速, 发送, 景区]",[今天 郑州 铁路 预计 发送 旅客 51.7 万人 管内 客流量 激增 中国 铁路 郑州 ...


# 可视化

In [58]:
# Arguments:
#           topics: A selection of topics to visualize
#           top_n_topics: Only select the top n most frequent topics
#           custom_labels: Whether to use custom topic labels that were defined using 
#                      `topic_model.set_topic_labels`.
#           title: Title of the plot.
#           width: The width of the figure.
#           height: The height of the figure.
topic_model.visualize_topics()


# 合并思路1：合并距离相近主题

In [59]:
topic_model.merge_topics(docs, [
  [8, 9],
  [0, 5, 10, 13],
  [1, 4],
  [2, 11, 12],
  [3, 6, 7]
])

topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,247,-1_景区_城市_河南_中国,"[景区, 城市, 河南, 中国, 游客, 历史, 洛阳市, 遗址, 博物馆, 发展]",[河南 多家 景区 陆续 发布 开园 公告 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续...
1,0,254,0_景区_发展_项目_建设,"[景区, 发展, 项目, 建设, 游客, 国家, 河南, 城市, 河南省, 洛阳市]",[行走 河南 读懂 中国 关注 全省 文旅文创 发展 大会 二十大 报告 指出 坚持 以文塑...
2,1,150,1_活动_景区_免费_时间,"[活动, 景区, 免费, 时间, 门票, 高速, 河南, 白云山, 体验, 广场]",[洛邑 福利社 中国 韵味 中国 范儿 洛邑 古城 中国 韵味 中国 范儿 大型 汉服 活动...
3,2,150,2_旅行_河南_王府_景区,"[旅行, 河南, 王府, 景区, 古城, 竹海, 历史, 地方, 中国, 泉州]",[五一 长假 就要 是不是 机会 出去 走走 放松 一下 喜欢 山水 风光 避开 人潮 地方...
4,3,131,3_中国_石窟_龙门石窟_艺术,"[中国, 石窟, 龙门石窟, 艺术, 世界, 位于, 文化遗产, 莫高窟, 造像, 朝代]",[旅行 洛阳 龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 ...
5,4,68,4_景区_旅客_郑州_年票,"[景区, 旅客, 郑州, 年票, 防控, 疫情, 开放, 高速, 客流, 客运]",[今天 郑州 铁路 预计 发送 旅客 51.7 万人 管内 客流量 激增 中国 铁路 郑州 ...


In [60]:
topic_model.visualize_topics()